# SQL script to read from customers silver tables and join them into one dim customers table

In [0]:
query = """
SELECT
    ROW_NUMBER () OVER (ORDER BY ci.customer_id) AS customer_key, -- surrogate key - unique key, faster for joins in reporting tools
    ci.customer_id,
    ci.customer_key AS customer_number,
    ci.first_name,
    ci.last_name,
    cl.country,
    ci.marital_status,
    CASE
    WHEN ci.gender <> 'n/a' THEN ci.gender
    ELSE COALESCE (ca.gender, 'n/a')
    END AS gender,
    ca.birth_date,
    ci.created_date AS create_date
FROM workspace.silver.crm_customers ci
LEFT JOIN workspace.silver.erp_customer_location cl
    ON ci.customer_key = cl.customer_number
LEFT JOIN workspace.silver.erp_cust_az12 ca
    ON CAST(ci.customer_id AS STRING) = CAST(ca.customer_id AS STRING)
"""
df = spark.sql(query)

In [0]:
df.limit(10).display()

# writing to gold table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_customers")

# sanity check for gold table

In [0]:
%sql
SELECT * FROM workspace.gold.dim_customers LIMIT 10